# Credit Card Fraud – Anomaly Detection & Explainability

## Loading the Data

In [ ]:
%pip -q install -U pandas pyarrow

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

data_path = Path("/content/drive/MyDrive/Colab Notebooks/MA/datasets/archive/creditcard.csv")

try:
    df = pd.read_csv(data_path)
    print(f"Successfully loaded data from: {data_path}")
    print(f"The dataset has {df.shape[0]} rows and {df.shape[1]} columns.")
except FileNotFoundError:
    print(f"ERROR: Data file not found at {data_path}. Please check the path.")


df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Successfully loaded data from: /content/drive/MyDrive/Colab Notebooks/MA/datasets/archive/creditcard.csv
The dataset has 284807 rows and 31 columns.


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## Preprocessing Pipeline

### Preprocessor for kernel-based models and autoencoders

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import cloudpickle
import pandas as pd
from pathlib import Path

actual_data_path = data_path
df_actual = pd.read_csv(actual_data_path)

y_full = df_actual["Class"].astype(int)
X_full = df_actual.drop(columns=["Class"])

numeric_cols = X_full.columns.tolist()
print("--- Columns determined for preprocessing ---")
print("Numeric cols:", numeric_cols)

numeric_pipe_kernel = Pipeline([
    ("scale", StandardScaler())
])

preprocessor_kernel = ColumnTransformer([
    ("num", numeric_pipe_kernel, numeric_cols)
])

print("\nFitting preprocessor_kernel...")
preprocessor_kernel.fit(X_full)
print("Preprocessor fitted.")

with open('/content/drive/MyDrive/Colab Notebooks/MA/Credit_Card_Fraud_ULB/preprocessors/preprocessor_kernel.pkl', 'wb') as f:
    cloudpickle.dump(preprocessor_kernel, f)
print("\n'preprocessor_kernel.pkl' saved successfully.")


# Build Preprocessing Pipeline for Autoencoders
numeric_pipe_autoenc = Pipeline([
    ("scale", MinMaxScaler())
])

preprocessor_autoenc = ColumnTransformer([
    ("num", numeric_pipe_autoenc, numeric_cols)
], sparse_threshold=0.0) # Force dense output for autoencoders

print("\nFitting preprocessor_autoenc...")
preprocessor_autoenc.fit(X_full)
print("Preprocessor fitted.")

with open('/content/drive/MyDrive/Colab Notebooks/MA/Credit_Card_Fraud_ULB/preprocessors/preprocessor_autoenc.pkl', 'wb') as f:
    cloudpickle.dump(preprocessor_autoenc, f)
print("\n'preprocessor_autoenc.pkl' saved successfully.")

--- Columns determined for preprocessing ---
Numeric cols: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']

Fitting preprocessor_kernel...
Preprocessor fitted.

'preprocessor_kernel.pkl' saved successfully.

Fitting preprocessor_autoenc...
Preprocessor fitted.

'preprocessor_autoenc.pkl' saved successfully.


### Preprocessors for tree-based models

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import cloudpickle
import pandas as pd
from pathlib import Path


df_actual = pd.read_csv(data_path)

# Separate features (X_full) from target (y_full)
y_full = df_actual["Class"].astype(int)
X_full = df_actual.drop(columns=["Class"])

# All features are numeric
numeric_cols = X_full.columns.tolist()

# Preprocessing Pipeline with StandardScaler
numeric_pipe_std = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler())
])

preprocessor_tree = ColumnTransformer([
    ("num", numeric_pipe_std, numeric_cols)
])

print("\nFitting preprocessor_std...")
preprocessor_tree.fit(X_full)
print("Preprocessor fitted.")

std_save_path = '/content/drive/MyDrive/Colab Notebooks/MA/Credit_Card_Fraud_ULB/preprocessors/preprocessor_tree.pkl'
with open(std_save_path, 'wb') as f:
    cloudpickle.dump(preprocessor_tree, f)
print(f"\n'{Path(std_save_path).name}' saved successfully.")


Fitting preprocessor_std...
Preprocessor fitted.

'preprocessor_tree.pkl' saved successfully.


## Saving final Products to Disk

In [ ]:
import pandas as pd
import cloudpickle
from pathlib import Path
import numpy as np
from sklearn.model_selection import train_test_split
import gc

PREPROCESSOR_FILES = [
    "preprocessor_kernel.pkl",
    "preprocessor_autoenc.pkl",
    "preprocessor_tree.pkl"
]
OUTPUT_DIR = Path('/content/drive/MyDrive/Colab Notebooks/MA/Credit_Card_Fraud_ULB/processed_data')
TARGET_COL = "Class"
CHUNK_SIZE = 50000

file_path = data_path
df = pd.read_csv(file_path)

y = df[TARGET_COL].astype(int)
X = df.drop(columns=[TARGET_COL])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Process and Save Data for each Preprocessor
for pkl_file in PREPROCESSOR_FILES:
    preprocessor_path = Path(pkl_file)
    print(f"\n--- Processing for: {preprocessor_path.name} ---")

    if not preprocessor_path.exists():
        print(f"WARNING: Skipping {preprocessor_path.name}, file not found.")
        continue

    base_name = preprocessor_path.stem.replace('preprocessor_', '')

    with open(preprocessor_path, "rb") as f:
        preprocessor = cloudpickle.load(f)

    # Process X_train -> Save -> Purge
    print(f"Processing X_train in chunks of {CHUNK_SIZE}...")
    train_chunks = [preprocessor.transform(X_train.iloc[i:i + CHUNK_SIZE]) for i in range(0, X_train.shape[0], CHUNK_SIZE)]
    X_train_transformed = np.vstack(train_chunks)

    print(f"Saving X_train_transformed for '{base_name}'...")
    np.save('/content/drive/MyDrive/Colab Notebooks/MA/Credit_Card_Fraud_ULB/processed_data/X_train_transformed_tree.npy', X_train_transformed)

    del train_chunks, X_train_transformed
    gc.collect()
    print(f"X_train_transformed for '{base_name}' saved and purged from memory.")

    print(f"\nProcessing X_test in chunks of {CHUNK_SIZE}...")
    test_chunks = [preprocessor.transform(X_test.iloc[i:i + CHUNK_SIZE]) for i in range(0, X_test.shape[0], CHUNK_SIZE)]
    X_test_transformed = np.vstack(test_chunks)

    print(f"Saving X_test_transformed for '{base_name}'...")
    np.save('/content/drive/MyDrive/Colab Notebooks/MA/Credit_Card_Fraud_ULB/processed_data/X_test_transformed_tree.npy', X_test_transformed)


--- Processing for: preprocessor_kernel.pkl ---
Processing X_train in chunks of 50000...
Saving X_train_transformed for 'kernel'...
X_train_transformed for 'kernel' saved and purged from memory.

Processing X_test in chunks of 50000...
Saving X_test_transformed for 'kernel'...

--- Processing for: preprocessor_autoenc.pkl ---
Processing X_train in chunks of 50000...
Saving X_train_transformed for 'autoenc'...
X_train_transformed for 'autoenc' saved and purged from memory.

Processing X_test in chunks of 50000...
Saving X_test_transformed for 'autoenc'...

--- Processing for: preprocessor_tree.pkl ---
